In [15]:
import numpy as np
import math

## 2.2 基础近邻指标

In [8]:
# CN相似度 (common neighbors)
def CN( set1, set2 ):
    return len( set1 & set2 )

# Jaccard相似度
def Jaccard( set1, set2 ):
    return len(set1&set2)/len(set1|set2)


In [9]:
a = { 1, 2, 3 }
b = { 2, 3, 4 }

print( CN(a,b) )
print( Jaccard(a,b) )

2
0.5


In [10]:
# 两个向量间的cos相似度
def cos4vector( v1, v2 ):
    return (np.dot(v1,v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))

# 两个集合间的cos相似度
def cos4set( set1, set2 ):
    return len(set1&set2)/(len(set1)*len(set2))**0.5

这里两个皮尔逊相关系数的定义考察定义，对于标准化后的向量：皮尔逊相似度等价于cos相似度

In [11]:
# 两个向量间的pearson相似度
def pearson( v1, v2 ):
    v1_mean = np.mean( v1 )
    v2_mean = np.mean( v2 )
    return ( np.dot( v1 - v1_mean, v2 - v2_mean) )/\
           ( np.linalg.norm( v1 - v1_mean ) *
             np.linalg.norm( v2 - v2_mean ) )

# 两个向量间的pearson相似度
def pearsonSimple( v1, v2 ):
    v1 -= np.mean( v1 )
    v2 -= np.mean( v2 )
    return cos4vector( v1, v2 )#调用余弦相似度函数

In [13]:
print( cos4set( a, b ) )

0.6666666666666666


In [14]:
a = [ 1, 3, 2 ]
b = [ 8, 9, 1 ]

print( cos4vector( a, b ) )
print( pearson( a, b ) )
print( pearsonSimple( a, b ) )

0.8183918171270462
0.11470786693528087
0.11470786693528087


## 2.5 进阶近邻指标
### 2.5.1 User-IIF和Item-IUF

User-IIF是在计算用户间相似度时，用流行度去衰减热门物品影响的相似权重。

逆物品频率(Inverse Item Frequency, IIF)是指物品频率的倒数，通常会取对数来使数据平滑，因为幂律函数取对数是一个线性函数。

In [ ]:
# 计算流行度
def getPopularity(data_sets):
    '''
    :param 用户或物品集合{iid:{uid1,uid2}}
    :return: 返回一个记录流行度的字典 {iid1: ppl1, iid2:ppl2}
    '''
    p = dict()
    for id in data_sets:
        frequency = len(data_sets[id])
        ppl = math.log1p(frequency)#即 ln(1+x)
        p[id] = ppl #得到流行度并记录起来
    return p

#IIF相似度
def getIIFSim(s1,s2,popularities):
    '''
    :param s1: 用户或物品集合 {iid1,iid2}
    :param s2: 用户或物品集合 {iid2,iid3}
    :param popularities: 流行度字典 {iid1: ppl1, iid2:ppl2}
    :return: IIF相似度
    '''
    s=0
    for i in s1 & s2:
        s += 1/popularities[i]
    return s/(len(s1)*len(s2))**0.5

### 2.5.2 更高效地利用流行度定义近邻指标

User-IIF和Item-IUF有一个遍历操作，以下方法可以提高计算的效率：

$$S_{x y}=\frac{|N(\boldsymbol{x}) \cap N(\boldsymbol{y})|}{|N(\boldsymbol{x})|^{1-\alpha_{y}} \times|N(\boldsymbol{y})|^{\alpha_{y}}} $$
$$\alpha=\frac{1+\operatorname{normalize}\left(\mathrm{ppl}_{y}\right)}{2}$$
$$\text { normalize }\left(\mathrm{ppl}_{y}\right)=\frac{\mathrm{ppl}_{y}}{\max (\mathrm{ppl})}$$
（可以使用Sigmoid函数来进行归一化来进一步化简）

考虑到最大流行度还是得统计到所有物品或用户的流行度后才能知道，而每天的点击量及浏览量会实时变化，在实际工作中统计全量数据的频率不会那么高，而Sigmoid的计算是不需要知道一个所谓的最大流行度的，所以Sigmoid可以当作局部更新时的归一化操作。且因为流行度不会为负，所以对流行度取Sigmoid值后范围是0.5~1所以可以直接令
$$\alpha =Sigmoid(pp_{y})$$

如果取Sigmoid后数据可能不平滑，也可以根据实际数据情况调整Sigmoid函数中$x$的系数。例如把调整后的Sigmoid公式写成：
$$Sigmoid^{\prime}(x)=\frac{{1}}{1+e^{-0.2x}}$$
在实际工作中可以通过机器学习训练系数。

In [ ]:
# 流行度的归一化
def normalizePopularities(popularities):
    '''
    :param popularities: 流行度字典 {iid1: ppl1, iid2:ppl2}
    :return: 归一化后的流行度字典 {iid1: ppl1, iid2:ppl2}
    '''
    maxp = max(popularities.values())
    norm_ppl = {}
    for k in popularities:
        norm_ppl[k] = popularities[k]/maxp
    return norm_ppl

# alpha相似度
def getAlphaSim( s1, s2, norm_ppl1,s2_index ):
    '''
    :param s1: 用户或物品集合 {uid1,uid2}
    :param s2: 用户或物品集合 {uid2,uid3}
    :param norm_ppl1: 归一化后的流行度字典 {iid1: ppl1, iid2:ppl2}
    :param s2_index: s2在数据集中的索引
    :return: alpha相似度
    '''
    alpha = ( 1 + norm_ppl1[s2_index] )/2
    return len(s1 & s2) / ( len(s1)**( 1 - alpha) * len(s2)**alpha)

#Sigmoid代码
def sigmoid(x):
    return 1/(1+math.e**(-x))